## Overfitting & Pruning

Decision Trees are prone to overfitting. If we have a lot of features and they all get used in building our tree, we will build a tree that perfectly represents our training data but is not general. A way to relax this is *pruning*. The idea is that we may not want to continue building the tree until all the leaves are pure (have only datapoints of one class). There are two main ways of pruning: *prepruning* and *postpruning*.

### Prepruning
*Prepruning* is making the decision tree algorithm stop early. Here are a few ways that we preprune:

* leaf size: Stop when the number of data points for a leaf gets below a threshold
* depth: Stop when the depth of the tree (distance from root to leaf) reaches a threshold
* mostly the same: Stop when some percent of the data points are the same (rather than all the same)
* error threshold: Stop when the error reduction (information gain) isn't improved significantly.

### Postpruning
As the name implies, *postpruning* involves building the tree first and then choosing to cut off some of the leaves (shorten some of the branches, the tree analogy really works well here).

Practically, the second approach of post-pruning overfit trees is more successful because it is not easy to precisely estimate when to stop growing the tree. 

Here's the psuedocode:

```
function Prune:
    if either left or right is not a leaf:
        call Prune on that split
    if both left and right are leaf nodes:
        calculate error associated with merging two nodes
        calculate error associated without merging two nodes
        if merging results in lower error:
            merge the leaf nodes
```

The important step of tree pruning is to define a criterion be used to determine the correct final tree size using one of the following methods:		
- Use a distinct dataset from the training set (called validation set), to evaluate the effect of post-pruning nodes from the tree.
- Build the tree by using the training set, then apply a statistical test to estimate whether pruning or expanding a particular node is likely to produce an improvement beyond the training set.
    - Error estimation
    - Significance testing (e.g., Chi-square test)
    
## Super Attributes

The information gain equation, $G(T,X)$ is biased toward attributes that have a large number of values over attributes that have a smaller number of values. These ‘Super Attributes’ will easily be selected as the root, resulted in a broad tree that classifies perfectly but performs poorly on unseen instances. We can penalize attributes with large numbers of values by using an alternative method for attribute selection, referred to as Gain Ratio.

![Step 4](images/super_attr.png)

## Ensembles

*Ensemble Methods* combine multiple machine learning algorithms to obtain better predictive performance. The idea is simple: run multiple models on the data and use their predictions to make a prediction that is better than any of the models could do alone.

In the case of decision trees, we extend this cognate to Random Forests


## Bagging

Bagging (developed by late Berkeley prof Leo Breiman), also known as *bootstrap aggregating*, is for running multiple models in parallel (the models don't use each other's results in order to predict). Each model gets a vote on the final prediction. The majority vote (again determined by a user-set parameter) is what is reported from the bagged model. Bagging is intended to address the problem of sample bias by enabling the model to capture bias within the training sample itself.


<center><img src="images/BaggingCropped_2.png" width="700"/></center>


For classification problems (predicting a categorical value), we choose the label with the most votes.

For regression problems (predicting a continuous value), we average the values given by all the models.

You can bag with any single algorithm or collection of algorithms, giving them each a vote to the final prediction. This means you can leverage the voting power of several different kinds of models, assuming that they have similar output space (what if they do not?). Scikits have a simple Bagging API implemented so that you can bag any [regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html) or [classifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html). [This example](http://scikit-learn.org/stable/auto_examples/ensemble/plot_bias_variance.html) is a useful one to study.


## Random Forests

Probably the most common ensemble method is a *Random Forest*, which consists of a collection of Decision Trees.

They were developed by the late Leo Breimen, who has the most extensive notes about them on his [webpage](http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm).

The idea is to repeatedly randomly select data from the dataset (*with replacement*) and build a Decision Tree with each new sample. The default is to have the randomly selected data be the same size as the initial dataset. Note that since we are sampling with replacement, many data points will be repeated in the sample and many won't be included.

Random Forests also limit each node of the Decision Tree to only consider splitting on a random subset of the features.

Here is the pseudocode for creating a Random Forest:

    CreateRandomForest(data, num_trees, num_features):
        Repeat num_trees times:
            Create a random sample of the test data with replacement
            Build a decision tree with that sample (only consider num_features features at each node)
        Return the list of the decision trees created

To classify a new document, use each tree to get a prediction. Choose the label that gets the most votes.

The default parameters that sklearn uses, which are also standard defaults, are 10 trees and only considering sqrt(m) features (where m is the total number of features).

### Out of Bag Error

We can analyze a Random Forest using the standard cross validation method of splitting the dataset into a training set and a testing set. However, if we're clever, we notice that each tree doesn't see all of the training data, so we can use the skipped data to cross validate each tree individually.

We'll skip the mathematical proof, but when selecting from the dataset, about one third of the data is left out (discussed [here](http://math.stackexchange.com/questions/203491/expected-coverage-after-sampling-with-replacement-k-times) if you want to think about the math). So every data point can be tested with about 1/3 of the trees. We calculate the percent of these that we get correct, and this is the *out-of-bag error*.

It has been proven that this is sufficient and that cross validation is not strictly necessary for a random forest, but we often still use it as that makes it easier to compare with other models.


### Feature Importance

We can use the random forest to determine which features are the most importance in predicting the class.

Breiman, the originator of random forests, uses out-of-bag error to determine feature importance, discussed [here](http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#varimp). The idea is to compare the out-of-bag error of the trees with the out-of-bag error of the trees if you change the feature's value (basically, if we screw with the value of the feature, how much does that impact the total error?). Here is the pseudocode for calculating the feature importance for a single feature:

        For every tree:
            Take the data that is not covered by the tree.
            Randomly permute the values of the feature (i.e. keep the same values,
                but shuffle them around the data points).
            Calculate the OOB error on the data with the feature values permuted.
            Subtract the permutated OOB from the OOB of the original data to get the
                feature importance on this tree.
        Average all the individual feature importances to get the feature importance.

sklearn uses a different method, described [here](http://scikit-learn.org/stable/modules/ensemble.html#feature-importance-evaluation). Their method doesn't involve using the out-of-bag score. Basically, the higher in the tree the feature is, the more important it is in determining the result of a data point. The expected fraction of data points that reach a node is used as an estimate of that feature's importance for that tree. Then average those values across all trees to get the feature's importance.


### Regression Forests

Random Forests can also be used for regression. They work very similarly to Classification Random Forests, except when predicting a new value, they take the average of the predicted values.


### Random Forests with sklearn

sklearn has both Random Forests for both [classification](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [regression](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). The documentation explains all of the parameters that you have control over, but you can use it like all other sklearn models:

```python
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifer()
rf.fit(X_train, y_train)
print "accuracy:", rf.score(X_test, y_test)
```

## Resources:

* [Random Forests by Leo Breiman and Adele Cutler](http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm)
* [sklearn ensembles](http://scikit-learn.org/stable/modules/ensemble.html)
* [Applied Data Science](http://columbia-applied-data-science.github.io/appdatasci.pdf) (Section 9.4.2)